In [ ]:
import boto3
import sagemaker
import json
import os
from utils import get_aws_profile_name, get_aws_iam_role
from datetime import datetime

LOCAL_EXECUTION = True

if LOCAL_EXECUTION:
    sess = boto3.Session(profile_name=get_aws_profile_name())
    sm = sess.client("sagemaker")
    iam = sess.client('iam')
    role = iam.get_role(RoleName=get_aws_iam_role())['Role']['Arn']
else:
    sess = boto3.Session()
    sm = sess.client("sagemaker")
    role = sagemaker.get_execution_role()

sagemaker_session = sagemaker.Session(boto_session=sess)
bucket = sagemaker_session.default_bucket()
prefix = "model-monitor-bring-your-own-model/"
region = sess.region_name

output_uri = "s3://{}/{}{}".format(bucket, prefix, "results-model-explain-"+datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))

In [ ]:
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    Processor,
)
from sagemaker.model_monitor.dataset_format import DatasetFormat
from utils import get_baseline_uri, get_dataset_uri


model_explainability_container_uri = sagemaker.image_uris.retrieve(
            framework="clarify",
            region=region,
            version="latest",
        )

        # Create the baseline job using
dataset_format = DatasetFormat.csv()

env = {
    "baseline_constraints": "/opt/ml/processing/baseline/constraints/analysis.json",
}

In [ ]:
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    Processor,
)

model_explainability_monitor_analyzer = Processor(
    image_uri=model_explainability_container_uri,
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    base_job_name=f"model-explainability-byom",
    sagemaker_session=sagemaker_session,
    max_runtime_in_seconds=1800,
    env=env,
)

model_explainability_monitor_analyzer.run(
    inputs=[ProcessingInput(
                source=get_baseline_uri('model-explainability-analysis'),
                destination="/opt/ml/processing/baseline/constraints",
                input_name="analysis_file"),
            ProcessingInput(
                source=get_baseline_uri('model-explainability-analysis_config'),
                destination="/opt/ml/processing/input/config",
                input_name="analysis_config"),
            ProcessingInput(
                source=get_dataset_uri('model-explainability-modified-data'),
                destination="/opt/ml/processing/input/data",
                input_name="dataset",),
            ],
    outputs=[
        ProcessingOutput(
                    source="/opt/ml/processing/output",
                    output_name="analysis_result",
                    destination=output_uri,
                )
    ],
)

In [ ]:
from utils import save_baseline
violations_uri = model_explainability_monitor_analyzer.latest_job.outputs[0].destination + '/constraint_violations.json'
save_baseline('model-explainability-violoations', violations_uri)

In [ ]:
try:
    json.loads(sagemaker.s3.S3Downloader().read_file(violations_uri, sagemaker_session=sagemaker_session))
except Exception as e:
    if (e.response['Error']['Code'] == "NoSuchKey"):
        print("No violations file found. Maybe No violations were detected")
    else:
        raise e
    